In [1]:
# --- Before your go ----
# 1. Rename Assignment-01-###.ipynb where ### is your student ID.
# 2. The deadline of Assignment-01 is 23:59pm, 03-23-2023


In [2]:
# In this assignment, you will 
# 1) explore Wikipedia text data
# 2) build language models
# 3) build a Naive Bayes classifier 


In [3]:
# Task0 Download the preprocessed data, enwiki-train.json 
# and enwiki-test from the Assignment-01 folder. In the data 
# file, each line contains a Wikipedia page with attributes, 
# title, label, and text. There are 1000 records in the train 
# file and 100 records in test file with ten categories.

# Your code

train_path="enwiki-train.json"
test_path="enwiki-test.json"    

with open(train_path) as f:
    train_list=f.readlines()
    print(train_list[0])
    print("train:",len(train_list))
with open(test_path) as f:
    test_list=f.readlines()
    print(test_list[0])
    print("test:",len(test_list))


{"title": "Citizen_Kane", "label": "Film", "text": "Citizen Kane is a 1941 American drama film produced by, directed by, and starring Orson Welles. He also co-wrote the screenplay with Herman J. Mankiewicz. The picture was Welles' first feature film. \"Citizen Kane\" is considered by many critics and experts to be the greatest film ever made. For 50 consecutive years, it stood at number 1 in the British Film Institute's \"Sight &amp; Sound\" decennial poll of critics, and it topped the American Film Institute's 100 Years\u00a0... 100 Movies list in 1998, as well as its 2007 update. The film was nominated for Academy Awards in nine categories and it won for Best Writing (Original Screenplay) by Mankiewicz and Welles. \"Citizen Kane\" is praised for Gregg Toland's cinematography, Robert Wise's editing, Bernard Herrmann's music, and its narrative structure, all of which have been considered innovative and precedent-setting.\nThe quasi-biographical film examines the life and legacy of Char

In [4]:
# Task1 --- Data exploring and preprocessing ---
# 1) Print out how many documents are in each class 
#    (for both train and test dataset)

# Your code

from collections import defaultdict

def cnt_class(json_path):
    with open(json_path) as f:
        json_list=f.readlines()
    dic=defaultdict(int)
    for item in json_list:
        label=eval(item)["label"]
        dic[label]+=1
    return dict(dic)

print("train:",cnt_class(train_path))
print("test:",cnt_class(test_path))


train: {'Film': 100, 'Book': 100, 'Politician': 100, 'Writer': 100, 'Food': 100, 'Actor': 70, 'Animal': 80, 'Software': 130, 'Artist': 100, 'Disease': 120}
test: {'Film': 10, 'Book': 10, 'Politician': 10, 'Writer': 10, 'Food': 10, 'Actor': 10, 'Animal': 10, 'Software': 10, 'Artist': 10, 'Disease': 10}


In [5]:
# Task1 --- Data exploring and preprocessing ---
# 2) Print out the average number of sentences in each class. 
# You may need to use sentence tokenization of NLTK.
# (for both train and test dataset)

# Your code

import nltk
import numpy as np

def avg_sen_num(json_path):
    with open(json_path) as f:
        json_list=f.readlines()
    dic=defaultdict(list)
    for item in json_list:
        label, text=eval(item)["label"], eval(item)["text"]
        num=len(nltk.sent_tokenize(text))
        dic[label].append(num)
    for i in dic:
        dic[i]=np.mean(dic[i])
    return dict(dic)

print("train:",avg_sen_num(train_path))
print("test:",avg_sen_num(test_path))

train: {'Film': 438.56, 'Book': 400.36, 'Politician': 706.2, 'Writer': 420.32, 'Food': 175.24, 'Actor': 76.7, 'Animal': 70.375, 'Software': 260.95384615384614, 'Artist': 306.47, 'Disease': 404.9}
test: {'Film': 364.7, 'Book': 295.9, 'Politician': 597.6, 'Writer': 294.9, 'Food': 107.6, 'Actor': 30.7, 'Animal': 46.8, 'Software': 160.1, 'Artist': 234.0, 'Disease': 311.7}


In [6]:
# Task1 --- Data exploring and preprocessing ---
# 3) Print out the average number of tokens in each class 
# (for both train and test dataset)

# Your code

def avg_token_num(json_path):
    with open(json_path) as f:
        json_list=f.readlines()
    dic=defaultdict(list)
    for item in json_list:
        label, text=eval(item)["label"], eval(item)["text"]
        num=len(nltk.word_tokenize(text))
        dic[label].append(num)   
    for i in dic:
        dic[i]=np.mean(dic[i])
    return dict(dic) 

print("train:",avg_token_num(train_path))
print("test:",avg_token_num(test_path))

train: {'Film': 11895.28, 'Book': 10540.51, 'Politician': 18644.3, 'Writer': 11849.91, 'Food': 3904.15, 'Actor': 1868.8428571428572, 'Animal': 1521.925, 'Software': 6302.3, 'Artist': 8212.91, 'Disease': 9322.958333333334}
test: {'Film': 9292.9, 'Book': 7711.1, 'Politician': 15204.3, 'Writer': 8499.4, 'Food': 2445.5, 'Actor': 677.5, 'Animal': 885.6, 'Software': 3972.8, 'Artist': 5706.4, 'Disease': 6988.8}


In [7]:
# Task1 --- Data exploring and preprocessing ---
# 4) For each sentence in the document, remove punctuations and 
# other special characters so that each sentence only contains 
# English words and numbers. To make your life easier, you can 
# make all words as lower cases. For each class, print out the 
# first article's name and the processed first 40 words.
# (for both train and test dataset)

# Your code

import re

def process_dataset(json_path):
    out=[]
    with open(json_path) as f:
        json_list=f.readlines()
    for item in json_list:
        dict=eval(item)
        new_text=[]
        for sent in nltk.sent_tokenize(dict["text"]):
            # new_text.append([c.lower() for c in nltk.word_tokenize(sent) if c.isalnum()])
            new_text.append([c.lower() for c in nltk.word_tokenize(sent) if not bool(re.match('^[!"#$%&\'()*+“”–,-.’/:;<=>?@[\\]^_`{|}~ ]+$', c))])
        dict["text"]=new_text
        out.append(dict)
    return out


def display(data):
    displayed=[]
    for i in data:
        if i["label"] in displayed:
            continue
        else:
            displayed.append(i["label"])
            cnt=0
            out=[]
            for sent in i["text"]:
                for word in sent:
                    # if(word!="<s>" and word !="</s>"):
                    cnt+=1
                    out.append(word)
                    if(cnt>=40):
                        break
            print(f'{i["title"]}: {out}')


print("train:")
train_data=process_dataset(train_path)
display(train_data)
print("test:")
test_data=process_dataset(test_path)
display(test_data)

train:
Citizen_Kane: ['citizen', 'kane', 'is', 'a', '1941', 'american', 'drama', 'film', 'produced', 'by', 'directed', 'by', 'and', 'starring', 'orson', 'welles', 'he', 'also', 'co-wrote', 'the', 'screenplay', 'with', 'herman', 'j.', 'mankiewicz', 'the', 'picture', 'was', 'welles', 'first', 'feature', 'film', 'citizen', 'kane', 'is', 'considered', 'by', 'many', 'critics', 'and', 'for', 'the', 'citizen', 'the', 'upon', 'after', 'he', 'although', 'following', 'principal', 'although', 'the', 'the', 'citizen', 'plot', 'in', 'holding', 'a', 'kane', 'thompson', 'he', 'thompson', 'through', 'in', 'she', 'while', 'when', 'by', 'he', 'kane', 'thompson', 'bernstein', 'kane', 'thompson', 'leland', 'both', 'kane', 'after', 'susan', 'kane', 'after', 'kane', 'when', 'thompson', 'back', 'they', 'they', 'cast', 'the', 'the', 'the', 'actor', 'pre-production', 'development', 'hollywood', 'he', 'in', 'although', 'following', 'rko', 'rko', 'throughout', 'welles', 'at', 'welles', 'welles', 'mercury', 'the'

In [8]:
# Task2 --- Build language models ---
# 1) Based on the training dataset, build unigram, bigram, 
# and trigram language models using Add-one smoothing technique.
# It is encouraged to implement models by yourself.

# Your code

# vocab默认选择freq>=5的word,不含<UNK>,<s>,</s>
# freq 含 unk </s> 不含 </s>

from nltk.probability import FreqDist
from math import log,exp
from tqdm import tqdm
import random

class unigram_model():
    def __init__(self, text_data, freq_limit=5):
        self.data=text_data
        self.vocab=[]
        self.freq=defaultdict(int)
        self.token_size=0
        self.build_vocab(freq_limit)
    
    def build_vocab(self,freq_limit):
        sent_list=[]
        for i in self.data:  # i[text]:  [[sent],[sent]]
            sent_list+=(i["text"] )
        freq=[]
        for i in sent_list:
            freq+=i
        self.token_size=len(freq)+len(sent_list) # 加上</s>的数量
        freq=FreqDist(freq)
        print("Building Vocalbulary...................")
        for i in tqdm(freq):
            if(freq[i]>=freq_limit):
                self.vocab.append(i)
                self.freq[i]+=freq[i]
            else:
                self.freq["<UNK>"]+=freq[i] 
        self.freq["</s>"]=len(sent_list)

    def unigram_prob(self, word): #add-one smoothing
        if word in self.vocab:
            return (self.freq[word]+1)/(self.token_size+len(self.vocab)+2) # for unk and </s>
        else:
            return (self.freq["<UNK>"]+1)/(self.token_size+len(self.vocab)+2)

    def unigram_sent_perplexity(self,sent):
        prob=0
        for word in sent:
            prob+=log(self.unigram_prob(word))
        prob+=log((self.freq["</s>"]+1)/(self.token_size+len(self.vocab)+2)) # 加上</s>的概率
        return (exp(prob/-(len(sent)+1))) # +1 for </s>

    def unigram_perplexity(self, test_data):
        sent_list=[]
        for i in test_data:  # i[text]:  [[sent],[sent]]
            sent_list+=i["text"] 
        perp=[]
        cnt=0
        print("Calculating perplexity...................")
        for sent in tqdm(sent_list):
            perp.append(self.unigram_sent_perplexity(sent))
        return np.mean(perp)
        
    def generate(self,max_len=30):
        out=["<s>"]
        s=sum(self.freq.values())
        while(out[-1]!="</s>" and len(out)<=max_len+1):
            c=random.randint(1,s)
            for f in self.freq:
                if(c<=self.freq[f]):
                    out.append(f)
                    break
                else:
                    c-=self.freq[f]
        return " ".join(out)


In [9]:
from nltk import ConditionalFreqDist

class bigram_model():
    def __init__(self, text_data, freq_limit=5):
        self.data=text_data
        self.vocab=[]
        self.freq=defaultdict(int)
        self.bifreq=ConditionalFreqDist()
        self.build_vocab(freq_limit)
    
    def build_vocab(self, freq_limit):
        sent_list=[]
        for i in self.data:  # i[text]:  [[sent],[sent]]
            sent_list+=(i["text"] )
        freq=[]
        for i in sent_list:
            freq+=i
        freq=FreqDist(freq)
        for i in freq:
            if(freq[i]>=freq_limit):
                self.vocab.append(i)
                self.freq[i]+=freq[i]
            else:
                self.freq["<UNK>"]+=freq[i] 
        self.freq["<s>"]=len(sent_list)
        print("Building Vocalbulary...................")
        for sent in tqdm(sent_list):
            bigram_list=list(nltk.ngrams(sent,2,pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))
            for bigram in bigram_list:
                self.bifreq[bigram[0]][bigram[1]]+=1

    def bigram_prob(self, bigram): #add-one smoothing
        if bigram[0] in self.vocab or bigram[0]=="<s>":
            if bigram[1] in self.vocab or bigram[1]=="</s>":
                return (self.bifreq[bigram[0]][bigram[1]]+1)/(self.freq[bigram[0]]+len(self.vocab)+2) # +2 for unk and </s>
            else:
                return (self.bifreq[bigram[0]]["<UNK>"]+1)/(self.freq[bigram[0]]+len(self.vocab)+2)
        else:
            if bigram[1] in self.vocab or bigram[1]=="</s>":
                return (self.bifreq["<UNK>"][bigram[1]]+1)/(self.freq["<UNK>"]+len(self.vocab)+2) # +2 for unk and </s>
            else:
                return (self.bifreq["<UNK>"]["<UNK>"]+1)/(self.freq["<UNK>"]+len(self.vocab)+2)

    def bigram_sent_perplexity(self,sent):
        prob=0
        for bigram in list(nltk.ngrams(sent,2,pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>')):
            prob+=log(self.bigram_prob(bigram))
        return exp(prob/-(len(sent)+1)) # +1 for </s>

    def bigram_perplexity(self, test_data):
        sent_list=[]
        for i in test_data:  # i[text]:  [[sent],[sent]]
            sent_list+=i["text"] 
        perp=[]
        cnt=0
        print("Calculating perplexity...................")
        for sent in tqdm(sent_list):
            perp.append(self.bigram_sent_perplexity(sent))
        return np.mean(perp)
        
    def generate(self,max_len=30):
        out=["<s>"]
        while(out[-1]!="</s>" and len(out)<=max_len+1):
            prev=out[-1]
            s=sum(self.bifreq[prev].values())
            c=random.randint(1,s)
            for key in self.bifreq[prev].keys():
                if(c<=self.bifreq[prev][key]):
                    out.append(key)
                    break
                else:
                    c-=self.bifreq[prev][key]
        return " ".join(out)

In [10]:
class trigram_model():
    def __init__(self, text_data, freq_limit=5):
        self.data=text_data
        self.vocab=[]
        self.freq=defaultdict(int)
        self.trifreq=ConditionalFreqDist()
        self.bifreq=defaultdict(int)
        self.build_vocab(freq_limit)
    
    def build_vocab(self, freq_limit):
        sent_list=[]
        for i in self.data:  # i[text]:  [[sent],[sent]]
            sent_list+=(i["text"] )
        freq=[]
        for i in sent_list:
            freq+=i
        freq=FreqDist(freq)
        for i in freq:
            if(freq[i]>=freq_limit):
                self.vocab.append(i)
                self.freq[i]+=freq[i]
            else:
                self.freq["<UNK>"]+=freq[i] 
        self.freq["<s>"]=len(sent_list)
        print("Building Vocalbulary...................")
        for sent in tqdm(sent_list):
            trigram_list=list(nltk.ngrams(sent,3,pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))[:-1]
            for trigram in trigram_list:
                self.trifreq[trigram[:2]][trigram[2]]+=1
                self.bifreq[trigram[:2]]+=1

    def trigram_prob(self, trigram): #add-one smoothing
        new=[]
        for i in trigram:
            if(i in self.vocab or i=="<s>" or i=="</s>"):
                new.append(i)
            else:
                new.append("<UNK>")
        new=tuple(new)
        return (self.trifreq[new[0:2]][new[2]]+1)/(self.bifreq[new[0:2]]+len(self.vocab)+2) # +2 for unk and </s>

    def trigram_sent_perplexity(self,sent):
        prob=0
        for trigram in list(nltk.ngrams(sent,3,pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))[:-1]:
            prob+=log(self.trigram_prob(trigram))
        return exp(prob/-(len(sent)+1)) # +1 for </s>

    def trigram_perplexity(self, test_data):
        sent_list=[]
        for i in test_data:  # i[text]:  [[sent],[sent]]
            sent_list+=i["text"] 
        perp=[]
        cnt=0
        print("Calculating perplexity...................")
        for sent in tqdm(sent_list):
            perp.append(self.trigram_sent_perplexity(sent))
        return np.mean(perp)
        
    def generate(self,max_len=30):
        out=["<s>","<s>"]
        while(out[-1]!="</s>" and len(out)<=max_len+2):
            prev=tuple(out[-2:])
            s=sum(self.trifreq[prev].values())
            c=random.randint(1,s)
            for key in self.trifreq[prev].keys():
                if(c<=self.trifreq[prev][key]):
                    out.append(key)
                    break
                else:
                    c-=self.trifreq[prev][key]
        return " ".join(out[1:])

In [11]:
# Task2 --- Build language models ---
# 2) Report the perplexity of these 3 trained models on the 
# testing dataset and explain your findings. 

# Your code

u_model_5=unigram_model(train_data,5)
print("unigram_perplexity:",u_model_5.unigram_perplexity(test_data))

b_model_5=bigram_model(train_data,5)
print("bigram_perplexity:",b_model_5.bigram_perplexity(test_data))

t_model_5=trigram_model(train_data,5)
print("trigram_perplexity:",t_model_5.trigram_perplexity(test_data))

Building Vocalbulary...................


100%|██████████| 180854/180854 [00:00<00:00, 1045754.23it/s]

Calculating perplexity...................



100%|██████████| 24440/24440 [00:45<00:00, 542.17it/s]


unigram_perplexity: 1524.4803520194803
Building Vocalbulary...................


...

bigram_perplexity: 6495.101228262545
Building Vocalbulary...................


100%|██████████| 338226/338226 [00:19<00:00, 17195.12it/s]


Calculating perplexity...................


100%|██████████| 24440/24440 [04:34<00:00, 89.07it/s] 

trigram_perplexity: 20251.165729692362


从以上结果可以观察到perplexity随着n的增大而增大，但在我们通常认知中n越大语言模型所拥有的信息更多，perplexity应该会更小，实验结果与我们的认知产生了矛盾。我推测这可能是因为训练集比较小导致的，数据集限制了更多元的语言模型对于语言的建模。更多元的语言模型可能往往需要更大的训练语料才能达到较好的效果。

In [15]:
# Task2 --- Build language models ---
# 3) Use each built model to generate five sentences and 
# explain these generated patterns.

# Your code

print("unigram model:")
for i in range(5):
    print(u_model_5.generate(40))

print("bigram model:")
for i in range(5):
    print(b_model_5.generate(40))

print("trigram model:")
for i in range(5):
    print(t_model_5.generate(40))


unigram model:
<s> primarily ramis of challenging teacher genre 2009 toothache the to full national militant and cutlets </s>
<s> but the removed did the menacingly formats fantastic so 30 supported kipling in because power political to sucrose arena government folklore cuba in screen lavien a from he 's <UNK> who small coat conventions the older real relations next the front
<s> practically quotations expedition locked plutonium subsidiary party </s>
<s> queen a viruses someone united at also gun the may come lady bedroom 's of contraceptives paradise </s>
<s> and loved othello from byproducts the extensively by sex various perfect economic actor 's 1918 tumor in later pushing </s>
bigram model:
<s> as a moscow </s>
<s> it leskov a significant </s>
<s> war against homosexuality he is a radio script resources to peek is pacing on the marchese di improvvisazione nuova </s>
<s> it might have ranked it was a close friend william archer daniels and his speeches three witches and ingredient

在一元语言模型中，词语之间没有关联，结束符的出现也十分莫名其妙。
在二元语言模型中，句子局部有一些关联，但整个句子仍没有体现较强的上下文关系。
在三元语言模型中，句子中词语的关联变得更加紧密，更像一句正常的句子。

In [13]:
# Task3 --- Build Naive Bayes classifiers --- 
# 1) Build Naive Bayes classifiers (with Laplace smoothing)

# Your code

# For unknown words, just ignore them
# don't consider <s> </s>

from collections import Counter

class Naive_Bayes_Model():
    def __init__(self,train_data,train_path):
        self.N_all_doc=len(train_data)
        # print("N_all_doc:", self.N_all_doc)

        self.N_doc=cnt_class(train_path)
        # print("N_doc:",self.N_doc)

        self.P_doc={}
        for cls in self.N_doc:
            self.P_doc[cls]=self.N_doc[cls]/self.N_all_doc
        # print("P_doc:",self.P_doc)

        self.dic_Naive=defaultdict(list)   #item: all docs with  class j
        for i in train_data:
            text,label=i["text"],i["label"]
            for sent in text:
                # print(label)
                self.dic_Naive[label]+=sent

        self.vocab_Naive=[]
        for i in self.dic_Naive:
            self.vocab_Naive+=self.dic_Naive[i]
        self.vocab_Naive=set(self.vocab_Naive)
        # print("vocab_size(without <UNK> <s> </s>):",len(vocab_Naive))

    def Naive_cls(self,text,k=1): # add-k smoothing
        dic=dict.fromkeys(['Film','Book','Politician','Writer','Food','Actor','Animal','Software','Artist','Disease'],0)
        for i in dic:
            dic[i]+=log(self.P_doc[i])
            counter=Counter(self.dic_Naive[i])
            for word in text:
                if word in counter:
                    dic[i]+=log((counter[word]+k)/(len(self.dic_Naive[i])+k*len(self.vocab_Naive)))
                elif word in self.vocab_Naive:
                    dic[i]+=log(k/(len(self.dic_Naive[i])+k*len(self.vocab_Naive)))
                else:  
                    pass 
        return max(dic,key=lambda x:dic[x])
    
    def Precision(self,l):
        # TP/TP+FP
        return l[0]/(l[0]+l[1]) if (l[0]+l[1])!=0 else 0 # modeified
        
    def Recall(self,l):
        # TP/TP+FN
        return l[0]/(l[0]+l[2])

    def F1_score(self,l):
        return 2*self.Precision(l)*self.Recall(l)/(self.Precision(l)+self.Recall(l)) if (self.Precision(l)+self.Recall(l))!=0 else 0
    
    def test(self,test_data,add_k=1):
        testset=[]
        for i in test_data:
            text,label=i["text"],i["label"]
            tmp=[]
            for sent in text:
                tmp+=sent
            testset.append([label,tmp])

        result=[(self.Naive_cls(text,add_k),label) for label,text in tqdm(testset)]

        macro_score=[]
        keylist=['Film','Book','Politician','Writer','Food','Actor','Animal','Software','Artist','Disease']
        micro_score=[0,0,0,0]
        for k in keylist:
            tmp=[0,0,0,0] # TP FP FN TN
            for pred,label in result:
                # if(label=="Actor"):
                #     print(pred)
                if(pred==k and label==k):
                    tmp[0]+=1
                    micro_score[0]+=1
                elif(pred==k and label!=k):
                    tmp[1]+=1
                    micro_score[1]+=1
                elif(pred!=k and label==k):
                    tmp[2]+=1
                    micro_score[2]+=1
                elif(pred!=k and label!=k):
                    tmp[3]+=1
                    micro_score[3]+=1
            F1=self.F1_score(tmp)
            macro_score.append(F1)
        micro_f1=self.F1_score(micro_score)
        macro_f1=np.mean(macro_score)
        print("micro_f1:",micro_f1)
        print("macro_f1:",macro_f1)
        print("f1_list:",list(zip(keylist,macro_score)))

In [14]:
# Task3 --- Build Naive Bayes classifiers --- 
# 2) Report Micro-F1 score and Macro-F1 score for these 
# classifiers on testing dataset explain our results.

# Your code

nbm=Naive_Bayes_Model(train_data,train_path)

nbm.test(test_data,add_k=1)

# nbm.test(test_data,add_k=0.25) # add-0.25效果更好，f1可以达到0.92，但我们并不能用testset来进行超参数选择。

100%|██████████| 100/100 [00:47<00:00,  2.09it/s]

micro_f1: 0.81
macro_f1: 0.7769310531230036
f1_list: [('Film', 0.6428571428571429), ('Book', 0.888888888888889), ('Politician', 0.8333333333333333), ('Writer', 0.8000000000000002), ('Food', 0.9473684210526316), ('Actor', 0), ('Animal', 0.8235294117647058), ('Software', 1.0), ('Artist', 1.0), ('Disease', 0.8333333333333333)]


在Add-One smoothing时，Actor 在训练集中数量较少并且容易被分错，分类准确率低，F1分数为0，拉低了宏平均，但微平均因为各类别进行相加，所以影响的幅度较小，所以导致了宏平均比微平均低一些。
